In [1]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import timedelta
import warnings


In [2]:
import os
os.getcwd()
path = "D:\\Romil\\Muthoot\\"


In [16]:
path = "D:\\Romil\\Muthoot\\"

df = pd.read_pickle( path + "DPD_allocation_scoring.pkl")
df.head()

,loanno,disbursaldate,dpd_sep2021,allocation_sep2021,dpd_oct2021,allocation_oct2021,dpd_nov2021,allocation_nov2021,dpd_dec2021,allocation_dec2021,...,dpd_jun2023,allocation_jun2023,dpd_jul2023,allocation_jul2023,dpd_aug2023,allocation_aug2023,dpd_sep2023,allocation_sep2023,dpd_oct2023,allocation_oct2023
0,TWL1123TN0000292041,18-Nov-2023,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,...,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable
1,TWL1123RJ0000291998,18-Nov-2023,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,...,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable
2,TWL1123RJ0000291773,18-Nov-2023,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,...,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable
3,TWL1123GJ0000291766,18-Nov-2023,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,...,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable
4,TWL1123KA0000292010,18-Nov-2023,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,...,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable


In [4]:
dpd_cols = []
allocation_cols = []
idcols = ['loanno','disbursaldate']
value_cols = list(set(df.columns) - set(idcols))
for col in value_cols:
    if "dpd" in col and "2019" not in col: 
        dpd_cols.append(col)
        df[df[col].isna()][col] =-99
        
    if "allocation" in col  and "2019" not in col: 
        allocation_cols.append(col)
        df[df[col].isna()][col] = "missing"



C:\Users\romil\AppData\Local\Temp\ipykernel_18216\3963019148.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df[col].isna()][col] =-99


In [5]:
df['disbursaldate'] = pd.to_datetime(df['disbursaldate'], format = "%d-%b-%Y")

In [6]:
df_2022 = df[df['disbursaldate'].dt.year >= 2019][idcols + allocation_cols]

allocation_stat = pd.melt(df_2022,id_vars = idcols ,value_vars = allocation_cols,
                          var_name = "Status_dt", value_name = "Allocation_status")
allocation_stat['Status_dt'] = allocation_stat['Status_dt'].str.replace('allocation_',"")
allocation_stat['Status_dt'] = pd.to_datetime(allocation_stat['Status_dt'] , format = "%b%Y" )
allocation_stat.head(2)

,loanno,disbursaldate,Status_dt,Allocation_status
0,TWL1123TN0000292041,2023-11-18,2022-09-01,Not Applicable
1,TWL1123RJ0000291998,2023-11-18,2022-09-01,Not Applicable


In [7]:
df_2022 = df[df['disbursaldate'].dt.year >= 2019][idcols + dpd_cols]

dpd_stat = pd.melt(df_2022,id_vars = idcols ,value_vars = dpd_cols,
                   var_name = "Status_dt", value_name = "DPD_status")

dpd_stat['Status_dt'] = dpd_stat['Status_dt'].str.replace('dpd_',"")
dpd_stat['Status_dt'] = pd.to_datetime(dpd_stat['Status_dt'] , format = "%b%Y" )

dpd_stat.head(3)

,loanno,disbursaldate,Status_dt,DPD_status
0,TWL1123TN0000292041,2023-11-18,2023-06-01,NaN
1,TWL1123RJ0000291998,2023-11-18,2023-06-01,NaN
2,TWL1123RJ0000291773,2023-11-18,2023-06-01,NaN


In [8]:
combined_table = allocation_stat.merge(dpd_stat , on = idcols.append("Status_dt") , how = "inner")
combined_table.head()

,loanno,disbursaldate,Status_dt,Allocation_status,DPD_status
0,TWL1123TN0000292041,2023-11-18,2022-09-01,Not Applicable,NaN
1,TWL1123RJ0000291998,2023-11-18,2022-09-01,Not Applicable,NaN
2,TWL1123RJ0000291773,2023-11-18,2022-09-01,Not Applicable,NaN
3,TWL1123GJ0000291766,2023-11-18,2022-09-01,Not Applicable,NaN
4,TWL1123KA0000292010,2023-11-18,2022-09-01,Not Applicable,NaN


In [9]:
combined_table['Status_dt'].max()

Timestamp('2023-10-01 00:00:00')

In [10]:
print(combined_table.shape)
combined_table = combined_table.drop_duplicates(subset=['loanno', 'Status_dt'], keep = 'first').reset_index(drop = True)
print(combined_table.shape)

combined_table = combined_table[combined_table['Status_dt'] > combined_table['disbursaldate']]
print(combined_table.shape)



(3450348, 5)
(3450348, 5)
(1718478, 5)


In [11]:
# Change the path
combined_table.to_pickle(path + "DPD_allocation_rev_scoring.pkl")

In [12]:
# Please be careful about the column NS_status in NACHStatus_scoring file

In [13]:
df = pd.read_pickle(path + "NACHStatus_scoring.pkl")

In [14]:
df.head()

,loanno,disbursaldate,customername,product,loanamount,tenure,onroadprice,servicechargewithtax,advanceemino,accirr,...,ns_jan2023,ns_feb2023,ns_mar2023,ns_apr2023,ns_may2023,ns_jun2023,ns_jul2023,ns_aug2023,ns_sep2023,ns_oct2023
0,TWL1023KA0000285133,26-Oct-2023,SHIVAKUMARA V C,Two Wheeler,72500.0,30.0,93336.0,856.0,0.0,24.05,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
1,TWL1023WB0000286473,28-Oct-2023,NIPA DUTTA,Two Wheeler,84000.0,12.0,110893.0,3469.0,2.0,17.36,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
2,TWL1023AP0000287704,31-Oct-2023,K MURALIDHAR,Two Wheeler,100000.0,30.0,111824.0,1180.0,0.0,24.04,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
3,TWL1123AP0000288581,04-Nov-2023,MIRZA NASAREY HASAN,Two Wheeler,78200.0,18.0,106564.0,1385.0,1.0,20.39,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
4,TWL1023MH0000288100,31-Oct-2023,VIKAS SUBHASH JADHAV,Two Wheeler,75600.0,18.0,94341.0,893.0,0.0,20.50,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable


In [15]:
df.columns

Index(['loanno', 'disbursaldate', 'customername', 'product', 'loanamount',
       'tenure', 'onroadprice', 'servicechargewithtax', 'advanceemino',
       'accirr', 'finrr', 'repaymentmode', 'assetmodel', 'assetclass',
       'manufacture', 'schemecode', 'bureaustatus', 'maindealercode',
       'subdealercode', 'business_source', 'ns_sep2021', 'ns_oct2021',
       'ns_nov2021', 'ns_dec2021', 'ns_jan2022', 'ns_feb2022', 'ns_mar2022',
       'ns_apr2022', 'ns_may2022', 'ns_jun2022', 'ns_jul2022', 'ns_aug2022',
       'ns_sep2022', 'ns_oct2022', 'ns_nov2022', 'ns_dec2022', 'ns_jan2023',
       'ns_feb2023', 'ns_mar2023', 'ns_apr2023', 'ns_may2023', 'ns_jun2023',
       'ns_jul2023', 'ns_aug2023', 'ns_sep2023', 'ns_oct2023'],
      dtype='object')

In [16]:
idcols = ['loanno', 'disbursaldate', 'customername', 'product', 'loanamount',
       'tenure', 'onroadprice', 'servicechargewithtax', 'advanceemino',
       'accirr', 'finrr', 'repaymentmode', 'assetmodel', 'assetclass',
       'manufacture', 'schemecode', 'bureaustatus', 'maindealercode',
       'subdealercode', 'business_source']
nscols = list(set(df.columns) - set(idcols))

ns_stat = pd.melt(df,id_vars = idcols ,value_vars = nscols,
                          var_name = "Status_dt", value_name = "NS_status")
ns_stat['Status_dt'] = ns_stat['Status_dt'].str.replace('ns_',"")
ns_stat['Status_dt'] = pd.to_datetime(ns_stat['Status_dt'] , format = "%b%Y" )
                                  
ns_stat.head(2)

,loanno,disbursaldate,customername,product,loanamount,tenure,onroadprice,servicechargewithtax,advanceemino,accirr,...,assetmodel,assetclass,manufacture,schemecode,bureaustatus,maindealercode,subdealercode,business_source,Status_dt,NS_status
0,TWL1023KA0000285133,26-Oct-2023,SHIVAKUMARA V C,Two Wheeler,72500.0,30.0,93336.0,856.0,0.0,24.05,...,HERO SPLENDOR PLUS SELF DRUM ALLOY FI i3S BSVI,Motor Cycle,Hero,1214.0,Positive,S1289,NaN,Dealer,2022-10-01,Not Applicable
1,TWL1023WB0000286473,28-Oct-2023,NIPA DUTTA,Two Wheeler,84000.0,12.0,110893.0,3469.0,2.0,17.36,...,YAMAHA RAY ZR 125 DISC DLX HYBRID RACING DARK ...,Scooter,Yamaha,1207.0,Positive,V0345,N0215,Dealer,2022-10-01,Not Applicable


In [ ]:
combined_table = pd.read_pickle(path + "DPD_allocation_rev_scoring.pkl")
combined_table['Status_dt'] = pd.to_datetime(combined_table['Status_dt'] , format = "%b%Y" )
combined_table['disbursaldate'] = pd.to_datetime(combined_table['disbursaldate']  )

ns_stat['Status_dt'] = pd.to_datetime(ns_stat['Status_dt'] , format = "%b%Y" )
ns_stat['disbursaldate'] = pd.to_datetime(ns_stat['disbursaldate']  )

join_cols = ['loanno','disbursaldate','Status_dt']
print(combined_table[join_cols].dtypes)
print(ns_stat[join_cols].dtypes)

combined_table_v2 = combined_table.merge(ns_stat , on = join_cols , how = "left")
print(combined_table.shape,combined_table_v2.shape)
combined_table_v2['NS_status'] = combined_table_v2['NS_status'].str.upper()


combined_table_v2['Bounce_status'] = np.where((combined_table_v2['NS_status'] != 'SUCCESS') & (combined_table_v2['NS_status'] != 'PAID BEFORE ALLOCATION') , 1,0)




In [ ]:
combined_table_v2['NS_status'].unique()

In [ ]:
combined_table_v2['NS_status'] = combined_table_v2['NS_status'].str.upper()
combined_table_v2['NS_status'].unique()

In [ ]:
print(combined_table_v2.shape)
combined_table_v2 = combined_table_v2.drop_duplicates(subset=['loanno', 'Status_dt'], keep = 'first').reset_index(drop = True)
print(combined_table_v2.shape)

combined_table_v2 = combined_table_v2[combined_table_v2['Status_dt'] > combined_table_v2['disbursaldate']]
print(combined_table.shape)


combined_table_v2.to_pickle(path + "DPD_allocation_nach_comb_scoring.pkl")